In [134]:
import numpy as np
import pandas as pd
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score

from sklearn.metrics import accuracy_score

from sklearn import tree as treeViz
import graphviz
from IPython.display import display
from dataset_reader import DataReader

In [57]:
file_name = "training_data_clean.csv"

def process_multiselect(series, target_tasks):
    """Convert multiselect strings to lists, keeping only specified features"""
    processed = []
    for response in series:
        if pd.isna(response) or response == '':
            processed.append([])
        else:
            # Check which of the target tasks are present in the response
            present_tasks = [task for task in target_tasks if task in str(response)]
            processed.append(present_tasks)
    return processed

def extract_rating(response):
    """
    Extract numeric rating from responses like '3 - Sometimes'.
    Returns None for missing responses
    """
    match = re.match(r'^(\d+)', str(response))
    return int(match.group(1)) if match else None

def visualize_tree(model, max_depth=5):
    """ 
    Generate and return an image representing an Sklearn decision tree.

    Each node in the visualization represents a node in the decision tree.
    In addition, visualization for each node contains:
        - The feature that is split on
        - The entropy (of the outputs `t`) at the node
        - The number of training samples at the node
        - The number of training samples with true/false values
        - The majority class (heart disease or not)
    The colour of the node also shows the majority class and purity

    See here: https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html

    Parameters:
        `model` - An Sklearn decision tree model
        `max_depth` - Max depth of decision tree to be rendered in the notebook.
         This is useful since the tree can get very large if the max_depth is
         set too high and thus making the resulting figure difficult to interpret.
    """
    dot_data = treeViz.export_graphviz(model,
                                       feature_names=target_tasks,
                                       max_depth=max_depth,
                                       class_names=['ChatGPT', 'Claude', 'Gemini'],
                                       filled=True,
                                       rounded=True)
    return display(graphviz.Source(dot_data))

In [58]:
df = pd.read_csv(file_name)
df.dropna(inplace=True)

In [151]:
target_tasks = [
        'Math computations',
        'Writing or debugging code',
        'Data processing or analysis', 
        'Explaining complex concepts simply',
        'Converting content between formats (e.g., LaTeX)',
        'Writing or editing essays/reports',
        'Drafting professional text (e.g., emails, résumés)',
        'Brainstorming or generating creative ideas'
    ]
best_tasks_lists = process_multiselect(df['Which types of tasks do you feel this model handles best? (Select all that apply.)'], target_tasks)
suboptimal_tasks_lists = process_multiselect(df['For which types of tasks do you feel this model tends to give suboptimal responses? (Select all that apply.)'], target_tasks)


mlb_best = MultiLabelBinarizer()
mlb_subopt = MultiLabelBinarizer()
    
best_tasks_encoded = mlb_best.fit_transform(best_tasks_lists)
suboptimal_tasks_encoded = mlb_subopt.fit_transform(suboptimal_tasks_lists)

# Use some rating features
academic_numeric = df['How likely are you to use this model for academic tasks?'].apply(extract_rating)
subopt_numeric = df['Based on your experience, how often has this model given you a response that felt suboptimal?'].apply(extract_rating)
# Combine features
X = np.hstack([academic_numeric.values.reshape(-1, 1), subopt_numeric.values.reshape(-1, 1), best_tasks_encoded, suboptimal_tasks_encoded])
y = df['label'].values

X_tv, X_test, y_tv, y_test = train_test_split(
    X,
    y,
    test_size=0.1,
    random_state=42,
    stratify=y
)

X_train, X_valid, y_train, y_valid = train_test_split(X_tv, y_tv, test_size=0.22, random_state=42)

print(X.shape)
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)


(734, 18)
(514, 18)
(146, 18)
(74, 18)


In [ ]:
"""dr = DataReader("training_data_clean.csv")
X, y = dr.to_numpy()
print(X.shape)
print(y.shape)
le = LabelEncoder()
y_encoded = le.fit_transform(y) 

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)
"""

(734, 6170)
(734,)
[0 1 1 ... 0 0 0]


In [153]:
n_estimators = [200, 300, 400, 500, 600, 700, 800]
#n_estimators = [700]

for i in n_estimators:
    print("Using " + str(i) + " trees:")
    model = RandomForestClassifier(
        n_estimators=i, 
        min_samples_split=20,
        min_samples_leaf=5,
        max_depth=10,
        random_state=42,
    )

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_valid_pred = model.predict(X_valid)
    y_test_pred  = model.predict(X_test)

    print("train acc:", accuracy_score(y_train, y_train_pred))
    print("valid acc:", accuracy_score(y_valid, y_valid_pred))
    print("test  acc:", accuracy_score(y_test, y_test_pred))

    skf = StratifiedKFold(n_splits=5, shuffle=True) #random_state removed

    cv_scores = cross_val_score(
        model,      
        X,           
        y,
        cv=skf,
        scoring="accuracy",
        n_jobs=-1
    )
    print("CV scores:", cv_scores)
    print("Mean CV acc:", cv_scores.mean())
    print("Std:", cv_scores.std())



#my stuff
#train_acc = model.score(X_train, y_train)
#test_acc = model.score(X_test, y_test)

#print(f"Training accuracy: {train_acc:.3f}")
#print(f"Test accuracy: {test_acc:.3f}")

Using 200 trees:
train acc: 0.7354085603112841
valid acc: 0.6506849315068494
test  acc: 0.7162162162162162
CV scores: [0.63265306 0.67346939 0.6122449  0.71428571 0.69178082]
Mean CV acc: 0.6648867766284596
Std: 0.03754300394485482
Using 300 trees:
train acc: 0.7315175097276264
valid acc: 0.6575342465753424
test  acc: 0.7162162162162162
CV scores: [0.63945578 0.70068027 0.70748299 0.63265306 0.66438356]
Mean CV acc: 0.6689311340974745
Std: 0.030658861537694926
Using 400 trees:
train acc: 0.7334630350194552
valid acc: 0.6575342465753424
test  acc: 0.7297297297297297
CV scores: [0.7414966  0.63945578 0.65306122 0.70068027 0.6369863 ]
Mean CV acc: 0.6743360357841767
Std: 0.04066647840474468
Using 500 trees:
train acc: 0.7373540856031129
valid acc: 0.6575342465753424
test  acc: 0.7162162162162162
CV scores: [0.72108844 0.70748299 0.68027211 0.6122449  0.64383562]
Mean CV acc: 0.6729848103625012
Std: 0.0402397826597134
Using 600 trees:
train acc: 0.7354085603112841
valid acc: 0.657534246575

In [ ]:
# Access the individual decision trees
for i, tree in enumerate(model.estimators_):
    print(tree.get_params())